# GraphRAG Implementation with LlamaIndex - V2

[GraphRAG (Graphs + Retrieval Augmented Generation)](https://www.microsoft.com/en-us/research/project/graphrag/) combines the strengths of Retrieval Augmented Generation (RAG) and Query-Focused Summarization (QFS) to effectively handle complex queries over large text datasets. While RAG excels in fetching precise information, it struggles with broader queries that require thematic understanding, a challenge that QFS addresses but cannot scale well. GraphRAG integrates these approaches to offer responsive and thorough querying capabilities across extensive, diverse text corpora.

This notebook provides guidance on constructing the GraphRAG pipeline using the LlamaIndex PropertyGraph abstractions using Neo4J.

This notebook updates the GraphRAG pipeline to v2. If you haven’t checked v1 yet, you can find it [here](https://github.com/run-llama/llama_index/blob/main/docs/docs/examples/cookbooks/GraphRAG_v1.ipynb). Following are the updates to the existing implementation:

1. Integrate with Neo4J Graph database.
2. Embedding based retrieval.



## Installation

`graspologic` is used to use hierarchical_leiden for building communities.

In [1]:
%pip install -U llama-index llama-index-graph-stores-neo4j graspologic numpy scipy future

  Using cached numpy-2.1.0-cp312-cp312-macosx_14_0_arm64.whl.metadata (60 kB)
  Using cached scipy-1.14.1-cp312-cp312-macosx_14_0_arm64.whl.metadata (60 kB)
Note: you may need to restart the kernel to use updated packages.


## Setup API Key and LLM

In [ ]:
import os
import openai as ai
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
ai.api_key = ""
ai.base_url = ""
os.environ["OPENAI_API_KEY"] = ai.api_key
os.environ["OPENAI_API_BASE"] = ai.base_url
Settings.llm = OpenAI(model="gpt-4")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-large", additional_kwargs={"encoding_format": "float"})
llm = Settings.llm

## GraphRAGExtractor

The GraphRAGExtractor class is designed to extract triples (subject-relation-object) from text and enrich them by adding descriptions for entities and relationships to their properties using an LLM.

This functionality is similar to that of the `SimpleLLMPathExtractor`, but includes additional enhancements to handle entity, relationship descriptions. For guidance on implementation, you may look at similar existing [extractors](https://docs.llamaindex.ai/en/latest/examples/property_graph/Dynamic_KG_Extraction/?h=comparing).

Here's a breakdown of its functionality:

**Key Components:**

1. `llm:` The language model used for extraction.
2. `extract_prompt:` A prompt template used to guide the LLM in extracting information.
3. `parse_fn:` A function to parse the LLM's output into structured data.
4. `max_paths_per_chunk:` Limits the number of triples extracted per text chunk.
5. `num_workers:` For parallel processing of multiple text nodes.


**Main Methods:**

1. `__call__:` The entry point for processing a list of text nodes.
2. `acall:` An asynchronous version of __call__ for improved performance.
3. `_aextract:` The core method that processes each individual node.


**Extraction Process:**

For each input node (chunk of text):
1. It sends the text to the LLM along with the extraction prompt.
2. The LLM's response is parsed to extract entities, relationships, descriptions for entities and relations.
3. Entities are converted into EntityNode objects. Entity description is stored in metadata
4. Relationships are converted into Relation objects. Relationship description is stored in metadata.
5. These are added to the node's metadata under KG_NODES_KEY and KG_RELATIONS_KEY.

**NOTE:** In the current implementation, we are using only relationship descriptions. In the next implementation, we will utilize entity descriptions during the retrieval stage.

In [3]:
import asyncio
import nest_asyncio

nest_asyncio.apply()

from typing import Any, List, Callable, Optional, Union, Dict
from IPython.display import Markdown, display

from llama_index.core.async_utils import run_jobs
from llama_index.core.indices.property_graph.utils import (
    default_parse_triplets_fn,
)
from llama_index.core.graph_stores.types import (
    EntityNode,
    KG_NODES_KEY,
    KG_RELATIONS_KEY,
    Relation,
)
from llama_index.core.llms.llm import LLM
from llama_index.core.prompts import PromptTemplate
from llama_index.core.prompts.default_prompts import (
    DEFAULT_KG_TRIPLET_EXTRACT_PROMPT,
)
from llama_index.core.schema import TransformComponent, BaseNode
from llama_index.core.bridge.pydantic import BaseModel, Field


class GraphRAGExtractor(TransformComponent):
    """Extract triples from a graph.

    Uses an LLM and a simple prompt + output parsing to extract paths (i.e. triples) and entity, relation descriptions from text.

    Args:
        llm (LLM):
            The language model to use.
        extract_prompt (Union[str, PromptTemplate]):
            The prompt to use for extracting triples.
        parse_fn (callable):
            A function to parse the output of the language model.
        num_workers (int):
            The number of workers to use for parallel processing.
        max_paths_per_chunk (int):
            The maximum number of paths to extract per chunk.
    """

    llm: LLM
    extract_prompt: PromptTemplate
    parse_fn: Callable
    num_workers: int
    max_paths_per_chunk: int

    def __init__(
        self,
        llm: Optional[LLM] = None,
        extract_prompt: Optional[Union[str, PromptTemplate]] = None,
        parse_fn: Callable = default_parse_triplets_fn,
        max_paths_per_chunk: int = 10,
        num_workers: int = 4,
    ) -> None:
        """Init params."""
        from llama_index.core import Settings

        if isinstance(extract_prompt, str):
            extract_prompt = PromptTemplate(extract_prompt)

        super().__init__(
            llm=llm or Settings.llm,
            extract_prompt=extract_prompt or DEFAULT_KG_TRIPLET_EXTRACT_PROMPT,
            parse_fn=parse_fn,
            num_workers=num_workers,
            max_paths_per_chunk=max_paths_per_chunk,
        )

    @classmethod
    def class_name(cls) -> str:
        return "GraphExtractor"

    def __call__(
        self, nodes: List[BaseNode], show_progress: bool = False, **kwargs: Any
    ) -> List[BaseNode]:
        """Extract triples from nodes."""
        return asyncio.run(
            self.acall(nodes, show_progress=show_progress, **kwargs)
        )

    async def _aextract(self, node: BaseNode) -> BaseNode:
        """Extract triples from a node."""
        assert hasattr(node, "text")

        text = node.get_content(metadata_mode="llm")
        try:
            llm_response = await self.llm.apredict(
                self.extract_prompt,
                text=text,
                max_knowledge_triplets=self.max_paths_per_chunk,
            )
            entities, entities_relationship = self.parse_fn(llm_response)
        except ValueError:
            entities = []
            entities_relationship = []

        existing_nodes = node.metadata.pop(KG_NODES_KEY, [])
        existing_relations = node.metadata.pop(KG_RELATIONS_KEY, [])
        entity_metadata = node.metadata.copy()
        for entity, entity_type, description in entities:
            entity_metadata["entity_description"] = description
            entity_node = EntityNode(
                name=entity, label=entity_type, properties=entity_metadata
            )
            existing_nodes.append(entity_node)

        relation_metadata = node.metadata.copy()
        for triple in entities_relationship:
            subj, obj, rel, description = triple
            relation_metadata["relationship_description"] = description
            rel_node = Relation(
                label=rel,
                source_id=subj,
                target_id=obj,
                properties=relation_metadata,
            )

            existing_relations.append(rel_node)

        node.metadata[KG_NODES_KEY] = existing_nodes
        node.metadata[KG_RELATIONS_KEY] = existing_relations
        return node

    async def acall(
        self, nodes: List[BaseNode], show_progress: bool = False, **kwargs: Any
    ) -> List[BaseNode]:
        """Extract triples from nodes async."""
        jobs = []
        for node in nodes:
            jobs.append(self._aextract(node))

        return await run_jobs(
            jobs,
            workers=self.num_workers,
            show_progress=show_progress,
            desc="Extracting paths from text",
        )

## GraphRAGStore

The `GraphRAGStore` class is an extension of the `Neo4jPropertyGraphStore`class, designed to implement GraphRAG pipeline. Here's a breakdown of its key components and functions:


The class uses community detection algorithms to group related nodes in the graph and then it generates summaries for each community using an LLM.


**Key Methods:**

`build_communities():`

1. Converts the internal graph representation to a NetworkX graph.

2. Applies the hierarchical Leiden algorithm for community detection.

3. Collects detailed information about each community.

4. Generates summaries for each community.

`generate_community_summary(text):`

1. Uses LLM to generate a summary of the relationships in a community.
2. The summary includes entity names and a synthesis of relationship descriptions.

`_create_nx_graph():`

1. Converts the internal graph representation to a NetworkX graph for community detection.

`_collect_community_info(nx_graph, clusters):`

1. Collects detailed information about each node based on its community.
2. Creates a string representation of each relationship within a community.

`_summarize_communities(community_info):`

1. Generates and stores summaries for each community using LLM.

`get_community_summaries():`

1. Returns the community summaries by building them if not already done.

In [4]:
import re
import networkx as nx
from graspologic.partition import hierarchical_leiden
from collections import defaultdict

from llama_index.core.llms import ChatMessage
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore


class GraphRAGStore(Neo4jPropertyGraphStore):
    community_summary = {}
    entity_info = None
    max_cluster_size = 5

    def generate_community_summary(self, text):
        """Generate summary for a given text using an LLM."""
        messages = [
            ChatMessage(
                role="system",
                content=(
                    "You are provided with a set of relationships from a knowledge graph, each represented as "
                    "entity1->entity2->relation->relationship_description. Your task is to create a summary of these "
                    "relationships. The summary should include the names of the entities involved and a concise synthesis "
                    "of the relationship descriptions. The goal is to capture the most critical and relevant details that "
                    "highlight the nature and significance of each relationship. Ensure that the summary is coherent and "
                    "integrates the information in a way that emphasizes the key aspects of the relationships."
                ),
            ),
            ChatMessage(role="user", content=text),
        ]
        response = llm.chat(messages)
        clean_response = re.sub(r"^assistant:\s*", "", str(response)).strip()
        print(clean_response)
        return clean_response

    def build_communities(self):
        """Builds communities from the graph and summarizes them."""
        nx_graph = self._create_nx_graph()
        community_hierarchical_clusters = hierarchical_leiden(
            nx_graph, max_cluster_size=self.max_cluster_size
        )
        self.entity_info, community_info = self._collect_community_info(
            nx_graph, community_hierarchical_clusters
        )
        self._summarize_communities(community_info)

    def _create_nx_graph(self):
        """Converts internal graph representation to NetworkX graph."""
        nx_graph = nx.Graph()
        triplets = self.get_triplets()
        for entity1, relation, entity2 in triplets:
            nx_graph.add_node(entity1.name)
            nx_graph.add_node(entity2.name)
            nx_graph.add_edge(
                relation.source_id,
                relation.target_id,
                relationship=relation.label,
                description=relation.properties["relationship_description"],
            )
        return nx_graph

    def _collect_community_info(self, nx_graph, clusters):
        """
        Collect information for each node based on their community,
        allowing entities to belong to multiple clusters.
        """
        entity_info = defaultdict(set)
        community_info = defaultdict(list)

        for item in clusters:
            node = item.node
            cluster_id = item.cluster

            # Update entity_info
            entity_info[node].add(cluster_id)

            for neighbor in nx_graph.neighbors(node):
                edge_data = nx_graph.get_edge_data(node, neighbor)
                if edge_data:
                    detail = f"{node} -> {neighbor} -> {edge_data['relationship']} -> {edge_data['description']}"
                    community_info[cluster_id].append(detail)

        # Convert sets to lists for easier serialization if needed
        entity_info = {k: list(v) for k, v in entity_info.items()}

        return dict(entity_info), dict(community_info)

    def _summarize_communities(self, community_info):
        """Generate and store summaries for each community."""
        for community_id, details in community_info.items():
            details_text = (
                "\n".join(details) + "."
            )  # Ensure it ends with a period
            self.community_summary[
                community_id
            ] = self.generate_community_summary(details_text)

    def get_community_summaries(self):
        """Returns the community summaries, building them if not already done."""
        if not self.community_summary:
            self.build_communities()
        return self.community_summary

/Users/jiangs5/Documents/RAG/distributed-rag/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## GraphRAGQueryEngine

The GraphRAGQueryEngine class is a custom query engine designed to process queries using the GraphRAG approach. It leverages the community summaries generated by the GraphRAGStore to answer user queries. Here's a breakdown of its functionality:

**Main Components:**

`graph_store:` An instance of GraphRAGStore, which contains the community summaries.
`llm:` A Language Model (LLM) used for generating and aggregating answers.


**Key Methods:**

`custom_query(query_str: str)`

1. This is the main entry point for processing a query. It retrieves community summaries, generates answers from each summary, and then aggregates these answers into a final response.

`generate_answer_from_summary(community_summary, query):`

1. Generates an answer for the query based on a single community summary.
Uses the LLM to interpret the community summary in the context of the query.

`aggregate_answers(community_answers):`

1. Combines individual answers from different communities into a coherent final response.
2. Uses the LLM to synthesize multiple perspectives into a single, concise answer.


**Query Processing Flow:**

1. Retrieve community summaries from the graph store.
2. For each community summary, generate a specific answer to the query.
3. Aggregate all community-specific answers into a final, coherent response.


**Example usage:**

```
query_engine = GraphRAGQueryEngine(graph_store=graph_store, llm=llm)

response = query_engine.query("query")
```

In [5]:
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.llms import LLM
from llama_index.core import PropertyGraphIndex

import re


class GraphRAGQueryEngine(CustomQueryEngine):
    graph_store: GraphRAGStore
    index: PropertyGraphIndex
    llm: LLM
    similarity_top_k: int = 20

    def custom_query(self, query_str: str) -> str:
        """Process all community summaries to generate answers to a specific query."""

        entities = self.get_entities(query_str, self.similarity_top_k)

        community_ids = self.retrieve_entity_communities(
            self.graph_store.entity_info, entities
        )
        community_summaries = self.graph_store.get_community_summaries()
        community_answers = [
            self.generate_answer_from_summary(community_summary, query_str)
            for id, community_summary in community_summaries.items()
            if id in community_ids
        ]

        final_answer = self.aggregate_answers(community_answers)
        return final_answer

    def get_entities(self, query_str, similarity_top_k):
        nodes_retrieved = self.index.as_retriever(
            similarity_top_k=similarity_top_k
        ).retrieve(query_str)

        enitites = set()
        pattern = r"(\w+(?:\s+\w+)*)\s*\({[^}]*}\)\s*->\s*([^(]+?)\s*\({[^}]*}\)\s*->\s*(\w+(?:\s+\w+)*)"

        for node in nodes_retrieved:
            matches = re.findall(pattern, node.text, re.DOTALL)

            for match in matches:
                subject = match[0]
                obj = match[2]
                enitites.add(subject)
                enitites.add(obj)

        return list(enitites)

    def retrieve_entity_communities(self, entity_info, entities):
        """
        Retrieve cluster information for given entities, allowing for multiple clusters per entity.

        Args:
        entity_info (dict): Dictionary mapping entities to their cluster IDs (list).
        entities (list): List of entity names to retrieve information for.

        Returns:
        List of community or cluster IDs to which an entity belongs.
        """
        community_ids = []

        for entity in entities:
            if entity in entity_info:
                community_ids.extend(entity_info[entity])

        return list(set(community_ids))

    def generate_answer_from_summary(self, community_summary, query):
        """Generate an answer from a community summary based on a given query using LLM."""
        prompt = (
            f"Given the community summary: {community_summary}, "
            f"how would you answer the following query? Query: {query}"
        )
        messages = [
            ChatMessage(role="system", content=prompt),
            ChatMessage(
                role="user",
                content="I need an answer based on the above information.",
            ),
        ]
        response = self.llm.chat(messages)
        cleaned_response = re.sub(r"^assistant:\s*", "", str(response)).strip()
        return cleaned_response

    def aggregate_answers(self, community_answers):
        """Aggregate individual community answers into a final, coherent response."""
        # intermediate_text = " ".join(community_answers)
        prompt = "Combine the following intermediate answers into a final, concise response."
        messages = [
            ChatMessage(role="system", content=prompt),
            ChatMessage(
                role="user",
                content=f"Intermediate answers: {community_answers}",
            ),
        ]
        final_response = self.llm.chat(messages)
        cleaned_final_response = re.sub(
            r"^assistant:\s*", "", str(final_response)
        ).strip()
        return cleaned_final_response

##  Build End to End GraphRAG Pipeline

Now that we have defined all the necessary components, let’s construct the GraphRAG pipeline:

1. Build a PropertyGraphIndex using `GraphRAGExtractor` and `GraphRAGStore`.
2. Create nodes/chunks from the text.
3. Construct communities and generate a summary for each community using the graph built above.
4. Create a `GraphRAGQueryEngine` and begin querying.

### Build ProperGraphIndex using `GraphRAGExtractor` and `GraphRAGStore`

In [6]:
KG_TRIPLET_EXTRACT_TMPL = """
-Goal-
Given a text document, identify all entities and their entity types from the text and all relationships among the identified entities.
Given the text, extract up to {max_knowledge_triplets} entity-relation triplets.

-Steps-
1. Identify all entities. For each identified entity, extract the following information:
- entity_name: Name of the entity, capitalized
- entity_type: Type of the entity
- entity_description: Comprehensive description of the entity's attributes and activities
Format each entity as ("entity"$$$$<entity_name>$$$$<entity_type>$$$$<entity_description>)

2. From the entities identified in step 1, identify all pairs of (source_entity, target_entity) that are *clearly related* to each other.
For each pair of related entities, extract the following information:
- source_entity: name of the source entity, as identified in step 1
- target_entity: name of the target entity, as identified in step 1
- relation: relationship between source_entity and target_entity
- relationship_description: explanation as to why you think the source entity and the target entity are related to each other

Format each relationship as ("relationship"$$$$"<source_entity>"$$$$"<target_entity>"$$$$"<relation>"$$$$"<relationship_description>")

3. When finished, output.

-Real Data-
######################
text: {text}
######################
output:"""

In [7]:
entity_pattern = r'\("entity"\$\$\$\$"(.+?)"\$\$\$\$"(.+?)"\$\$\$\$"(.+?)"\)'
relationship_pattern = r'\("relationship"\$\$\$\$"(.+?)"\$\$\$\$"(.+?)"\$\$\$\$"(.+?)"\$\$\$\$"(.+?)"\)'


def parse_fn(response_str: str) -> Any:
    entities = re.findall(entity_pattern, response_str)
    relationships = re.findall(relationship_pattern, response_str)
    print(response_str, entities, relationships)
    return entities, relationships


kg_extractor = GraphRAGExtractor(
    llm=llm,
    extract_prompt=KG_TRIPLET_EXTRACT_TMPL,
    max_paths_per_chunk=2,
    parse_fn=parse_fn,
)

## Docker Setup And Neo4J setup

To launch Neo4j locally, first ensure you have docker installed. Then, you can launch the database with the following docker command.

```
docker run \
    -p 7474:7474 -p 7687:7687 \
    -v $PWD/data:/data -v $PWD/plugins:/plugins \
    --name neo4j-apoc \
    -e NEO4J_apoc_export_file_enabled=true \
    -e NEO4J_apoc_import_file_enabled=true \
    -e NEO4J_apoc_import_file_use__neo4j__config=true \
    -e NEO4JLABS_PLUGINS=\[\"apoc\"\] \
    neo4j:latest
```
From here, you can open the db at http://localhost:7474/. On this page, you will be asked to sign in. Use the default username/password of neo4j and neo4j.

Once you login for the first time, you will be asked to change the password.

In [8]:
# Note: used to be `Neo4jPGStore`
graph_store = GraphRAGStore(
    username="neo4j", password="subway-zebra-modest-salute-john-1136", url="bolt://localhost:7687"
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"


## Load Data

We will use a sample news article dataset retrieved from Diffbot, which Tomaz has conveniently made available on GitHub for easy access.

The dataset contains 2,500 samples; for ease of experimentation, we will use 50 of these samples, which include the `title` and `text` of news articles.

In [9]:
import pandas as pd
from llama_index.core import Document

news = pd.read_csv(
    "https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/news_articles.csv"
)[:50]

documents = [
    Document(text=f"{row['title']}: {row['text']}")
    for i, row in news.iterrows()
]

news.head()

,title,date,text
0,Chevron: Best Of Breed,2031-04-06T01:36:32.000000000+00:00,JHVEPhoto Like many companies in the O&G secto...
1,FirstEnergy (NYSE:FE) Posts Earnings Results,2030-04-29T06:55:28.000000000+00:00,FirstEnergy (NYSE:FE – Get Rating) posted its ...
2,Dáil almost suspended after Sinn Féin TD put p...,2023-06-15T14:32:11.000000000+00:00,The Dáil was almost suspended on Thursday afte...
3,Epic’s latest tool can animate hyperrealistic ...,2023-06-15T14:00:00.000000000+00:00,"Today, Epic is releasing a new tool designed t..."
4,"EU to Ban Huawei, ZTE from Internal Commission...",2023-06-15T13:50:00.000000000+00:00,The European Commission is planning to ban equ...


Prepare documents as required by LlamaIndex

### Create nodes/ chunks from the text.

In [10]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(
    chunk_size=1024,
    chunk_overlap=20,
)
nodes = splitter.get_nodes_from_documents(documents)
len(nodes)

50

In [11]:
from llama_index.core import PropertyGraphIndex
index = PropertyGraphIndex.from_existing(
    property_graph_store=graph_store,
    kg_extractors=[kg_extractor],
    show_progress=True,
)

In [12]:
# import asyncio
# from functools import wraps, partial

# def async_wrap(func):
#     @wraps(func)
#     async def run(*args, loop=None, executor=None, **kwargs):
#         if loop is None:
#             loop = asyncio.get_event_loop()
#         pfunc = partial(func, *args, **kwargs)
#         return await loop.run_in_executor(executor, pfunc)
#     return run

# async_insert_nodes = async_wrap(index.insert_nodes)

# async def insert_nodes(ns):
#     await asyncio.gather(*[async_insert_nodes([n]) for n in ns])

# for i in range(0, len(nodes), 10):
#     print(f"Processing nodes {i} to {i + 10}")
#     end = i + 10 if i + 10 < len(nodes) else len(nodes)
#     asyncio.run(insert_nodes(nodes[i:end]))
index.insert_nodes(nodes)

Extracting paths from text:   2%|▏         | 1/50 [00:18<15:08, 18.54s/it]

 ```
("entity"$$$$"Stellantis"$$$$"Company"$$$$"An automaker formed by the merger of Fiat Chrysler and France’s PSA Peugeot.")
("entity"$$$$"Belvidere Assembly Plant"$$$$"Facility"$$$$"An assembly plant located in Belvidere, Illinois, that produces the Jeep Cherokee.")
("entity"$$$$"Jeep Cherokee"$$$$"Vehicle"$$$$"A mid-size SUV produced by Stellantis at the Belvidere Assembly Plant.")
("entity"$$$$"Gov. J.B. Pritzker"$$$$"Person"$$$$"The Governor of Illinois who has assembled a response team to help displaced workers.")
("entity"$$$$"Department of Commerce and Economic Opportunity"$$$$"Government Agency"$$$$"A government agency in Illinois that is helping displaced workers find new employment.")

("relationship"$$$$"Stellantis"$$$$"Belvidere Assembly Plant"$$$$"Owns"$$$$"Stellantis owns the Belvidere Assembly Plant and has decided to close it.")
("relationship"$$$$"Belvidere Assembly Plant"$$$$"Jeep Cherokee"$$$$"Produces"$$$$"The Belvidere Assembly Plant produces the Jeep Cherokee.")

Extracting paths from text:   4%|▍         | 2/50 [00:22<08:06, 10.14s/it]

 ```
("entity"$$$$"Anil Goteti"$$$$"Person"$$$$"Former senior executive at Flipkart and founder of Scapia and Protonn")
("entity"$$$$"Scapia"$$$$"Company"$$$$"A travel fintech company offering a credit card and app for earning and redeeming travel rewards")
("entity"$$$$"Flipkart"$$$$"Company"$$$$"An e-commerce company where Anil Goteti previously worked as a senior executive")
("entity"$$$$"Federal Bank"$$$$"Bank"$$$$"A banking institution partnering with Scapia for the credit card")
("entity"$$$$"VISA"$$$$"Payment Network"$$$$"A global payment network on which the Scapia credit card is based")
("entity"$$$$"Protonn"$$$$"Company"$$$$"Anil Goteti's earlier venture that shut down and returned investments")
("entity"$$$$"Matrix Partners"$$$$"Investment Firm"$$$$"An investment firm that backed Protonn")
("entity"$$$$"Binny Bansal"$$$$"Person"$$$$"Co-founder of Flipkart and investor in Protonn")

("relationship"$$$$"Anil Goteti"$$$$"Scapia"$$$$"Founder"$$$$"Anil Goteti is the founder of Sc

Extracting paths from text:   6%|▌         | 3/50 [00:28<06:10,  7.89s/it]

...

Extracting paths from text:  96%|█████████▌| 48/50 [06:44<00:18,  9.18s/it]

 ```
("entity"$$$$"XPeng"$$$$"Company"$$$$"Chinese electric-vehicle maker that developed assisted-driving technology")
("entity"$$$$"Tesla"$$$$"Company"$$$$"Electric-vehicle maker known for its self-driving technology")
("entity"$$$$"Beijing"$$$$"City"$$$$"Capital city of China where XPeng's assisted-driving technology was launched")
("entity"$$$$"Stock"$$$$"Financial Instrument"$$$$"Represents ownership in a company, in this case, XPeng's stock was rising in premarket trading")

("relationship"$$$$"XPeng"$$$$"Tesla"$$$$"Rival"$$$$"XPeng is described as a rival to Tesla in the electric-vehicle market")
("relationship"$$$$"XPeng"$$$$"Beijing"$$$$"Launch Location"$$$$"XPeng launched its assisted-driving technology in Beijing")
``` [('XPeng', 'Company', 'Chinese electric-vehicle maker that developed assisted-driving technology'), ('Tesla', 'Company', 'Electric-vehicle maker known for its self-driving technology'), ('Beijing', 'City', "Capital city of China where XPeng's assisted-driving t

Extracting paths from text:  96%|█████████▌| 48/50 [06:54<00:17,  8.63s/it]


APITimeoutError: Request timed out.

 ```
("entity"$$$$"Newcastle United"$$$$"Football Club"$$$$"A professional football club based in Newcastle upon Tyne, England, that competes in the Premier League.")
("entity"$$$$"Premier League"$$$$"Football League"$$$$"The highest level of the English football league system.")
("entity"$$$$"Eddie Howe"$$$$"Football Manager"$$$$"The manager of Newcastle United Football Club.")
("entity"$$$$"Aston Villa"$$$$"Football Club"$$$$"A professional football club based in Aston, Birmingham, England, that competes in the Premier League.")
("entity"$$$$"Unai Emery"$$$$"Football Manager"$$$$"The manager of Aston Villa Football Club.")
("entity"$$$$"Brentford"$$$$"Football Club"$$$$"A professional football club based in Brentford, West London, England, that competes in the Premier League.")
("entity"$$$$"Manchester City"$$$$"Football Club"$$$$"A professional football club based in Manchester, England, that competes in the Premier League.")
("entity"$$$$"Liverpool"$$$$"Football Club"$$$$"A profess

In [13]:
index.property_graph_store.get_triplets()

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (e, e) { ... }} {position: line: 3, column: 9, offset: 47} for query: "MATCH (e:`__Entity__`) \n        WITH e\n        CALL {\n            WITH e\n            MATCH (e)-[r]->(t:`__Entity__`)\n            RETURN e.name AS source_id, [l in labels(e) WHERE NOT l IN ['__Entity__', '__Node__'] | l][0] AS source_type,\n                   e{.* , embedding: Null, name: Null} AS source_properties,\n                   type(r) AS type,\n                   r{.*} AS rel_properties,\n                   t.name AS target_id, [l in labels(t) WHERE NOT l IN ['__Entity__', '__Node__'] | l][0] AS target_type,\n                   t{.* , embedding: Null, name: Null} AS target_properties\n         

[[EntityNode(label='Company', embedding=None, properties={'id': 'Chevron', 'entity_description': 'A multinational energy corporation headquartered in San Ramon, California, USA, involved in oil and gas exploration, production, and refining.', 'triplet_source_id': 'ec61a0ac-3b6f-458b-a095-8d3b93b27121'}, name='Chevron'),
  Relation(label='Represented By', source_id='Chevron', target_id='NYSE:CVX', properties={'relationship_description': 'Chevron is represented by the stock ticker symbol NYSE:CVX on the New York Stock Exchange.', 'triplet_source_id': 'ca6982c3-5e32-4186-81c9-60c21cff3eeb'}),
  EntityNode(label='Stock Ticker', embedding=None, properties={'id': 'NYSE:CVX', 'entity_description': 'The stock ticker symbol for Chevron on the New York Stock Exchange.', 'triplet_source_id': 'ec61a0ac-3b6f-458b-a095-8d3b93b27121'}, name='NYSE:CVX')],
 [EntityNode(label='Company', embedding=None, properties={'id': 'Chevron', 'entity_description': 'A multinational energy corporation headquartered i

In [14]:
index.property_graph_store.get_triplets()[10][0].properties

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (e, e) { ... }} {position: line: 3, column: 9, offset: 47} for query: "MATCH (e:`__Entity__`) \n        WITH e\n        CALL {\n            WITH e\n            MATCH (e)-[r]->(t:`__Entity__`)\n            RETURN e.name AS source_id, [l in labels(e) WHERE NOT l IN ['__Entity__', '__Node__'] | l][0] AS source_type,\n                   e{.* , embedding: Null, name: Null} AS source_properties,\n                   type(r) AS type,\n                   r{.*} AS rel_properties,\n                   t.name AS target_id, [l in labels(t) WHERE NOT l IN ['__Entity__', '__Node__'] | l][0] AS target_type,\n                   t{.* , embedding: Null, name: Null} AS target_properties\n         

{'id': 'Chevron',
 'entity_description': 'A multinational energy corporation headquartered in San Ramon, California, USA, involved in oil and gas exploration, production, and refining.',
 'triplet_source_id': 'ec61a0ac-3b6f-458b-a095-8d3b93b27121'}

In [15]:
index.property_graph_store.get_triplets()[10][1].properties

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (e, e) { ... }} {position: line: 3, column: 9, offset: 47} for query: "MATCH (e:`__Entity__`) \n        WITH e\n        CALL {\n            WITH e\n            MATCH (e)-[r]->(t:`__Entity__`)\n            RETURN e.name AS source_id, [l in labels(e) WHERE NOT l IN ['__Entity__', '__Node__'] | l][0] AS source_type,\n                   e{.* , embedding: Null, name: Null} AS source_properties,\n                   type(r) AS type,\n                   r{.*} AS rel_properties,\n                   t.name AS target_id, [l in labels(t) WHERE NOT l IN ['__Entity__', '__Node__'] | l][0] AS target_type,\n                   t{.* , embedding: Null, name: Null} AS target_properties\n         

{'relationship_description': 'Chevron is traded on the New York Stock Exchange under the ticker symbol NYSE:CVX.',
 'triplet_source_id': 'ec61a0ac-3b6f-458b-a095-8d3b93b27121'}

### Build communities

This will create communities and summary for each community.

In [16]:
index.property_graph_store.build_communities()

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (e, e) { ... }} {position: line: 3, column: 9, offset: 47} for query: "MATCH (e:`__Entity__`) \n        WITH e\n        CALL {\n            WITH e\n            MATCH (e)-[r]->(t:`__Entity__`)\n            RETURN e.name AS source_id, [l in labels(e) WHERE NOT l IN ['__Entity__', '__Node__'] | l][0] AS source_type,\n                   e{.* , embedding: Null, name: Null} AS source_properties,\n                   type(r) AS type,\n                   r{.*} AS rel_properties,\n                   t.name AS target_id, [l in labels(t) WHERE NOT l IN ['__Entity__', '__Node__'] | l][0] AS target_type,\n                   t{.* , embedding: Null, name: Null} AS target_properties\n         

### Summary of Relationships

**Chevron:**
- **Stock Listing:** Chevron is listed on the New York Stock Exchange (NYSE) under the ticker symbol CVX.
- **Earnings Estimates:** Chevron has Q2 consensus earnings estimates that have risen sharply.
- **Industry Sector:** Chevron operates within the Oil & Gas (O&G) sector.

**FirstEnergy:**
- **Stock Listing:** FirstEnergy is listed on the NYSE stock exchange.
- **Earnings Report:** RTT News reported on FirstEnergy's earnings results.

**State Street:**
- **Stock Listing:** State Street is listed on the New York Stock Exchange (NYSE).
- **Analyst Coverage:** State Street is covered by analysts who provide future growth predictions.
- **Total Shareholder Return (TSR):** State Street shareholders have received a TSR of 22% over one year.
- **Earnings Per Share (EPS):** State Street's EPS has improved by 6.1% per year despite the share price slip.
- **Insider Transactions:** Investors check data on insider transactions related to State Street.


### Create QueryEngine

In [17]:
query_engine = GraphRAGQueryEngine(
    graph_store=index.property_graph_store,
    llm=llm,
    index=index,
    similarity_top_k=10,
)

### Querying

In [18]:
response = query_engine.query(
    "What are the main news discussed in the document?"
)
display(Markdown(f"{response.response}"))

The main news discussed in the document includes:

1. **Chevron**:
   - Chevron's Q2 consensus earnings estimates have risen sharply.

2. **FirstEnergy**:
   - RTT News reported on FirstEnergy's earnings results.

3. **State Street**:
   - State Street shareholders have received a Total Shareholder Return (TSR) of 22% over one year.
   - State Street's Earnings Per Share (EPS) has improved by 6.1% per year despite the share price slip.
   - State Street's revenue has increased by 0.5% over the time period.
   - Investors are checking data on insider transactions related to State Street.

4. **Uber Technologies**:
   - Uber operates its mobility business in Israel and Uber Eats in Italy, competing with services like Deliveroo, Glovo, and Just Eat Takeaway.com.
   - Uber partners with IT Taxi in Italy.
   - Dara Khosrowshahi is the CEO, and the company is based in San Francisco.
   - Mauro Orru wrote an article about Uber Technologies.

5. **Delta Air Lines**:
   - The pandemic led to significant financial strain, resulting in the suspension of its quarterly dividend in March 2020.
   - Delta received $11.9 billion in U.S. government assistance with restrictions on dividends and stock repurchases.
   - Delta resumed its quarterly dividend in June 2022, marking a milestone in its recovery.

These points summarize the key updates and financial metrics mentioned for each company.

In [19]:
response = query_engine.query("What are the main news in energy sector?")
display(Markdown(f"{response.response}"))

The main news in the energy sector includes:

1. **Chevron**: Q2 consensus earnings estimates have risen sharply, indicating positive financial performance expectations.
2. **FirstEnergy**: RTT News reported on FirstEnergy's earnings results, highlighting the company's financial performance updates.
3. **XPeng**:
   - Launched self-driving and assisted-driving technologies, starting in Beijing.
   - Recognized as a rival to Tesla in the electric vehicle market.
4. **Tesla**:
   - Facing competition from XPeng and scrutiny from short sellers.
   - Partnered with General Motors to expand Supercharger network access.
   - Under market analysis by experts like Ihor Dusaniwsky from S3 Partners.
5. **GE Vernova**:
   - Supplying wind turbines for Amplus Solar's project in Tamil Nadu.
   - Key facilities include a technology centre in Bengaluru and a manufacturing facility in Pune.
   - Introduced the 2.7-132 Wind Turbine.
   - Deepak Maloo appointed as Regional Sales Leader for the Asia Pacific region.
6. **Amplus Solar**:
   - Expanding renewable energy portfolio with a wind power project in Tamil Nadu.
   - Strategic partnership with Petronas.
   - Led by CEO Sharad Pungalia.

These updates highlight significant developments in the energy sector, including financial performance, technological advancements, strategic partnerships, and leadership roles.